In [ ]:
# prompt: code pour se connecter a mon drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
pip install loguru

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.8 MB/s eta 0:00:00


In [ ]:
from loguru import logger
from os import environ
from pathlib import Path

In [ ]:
ROOT_DIR = '/content/drive/MyDrive'  # Changed to a subfolder within MyDrive
# or ROOT_DIR = '/content/drive/Shareddrives/YOUR_SHARED_DRIVE_NAME' if using a shared drive
DATA_DIR = Path(ROOT_DIR, "data_ner")  # data
CONFIG_DIR = Path(ROOT_DIR, "config_ner")  # config, bins
OUTPUT_DIR  = Path(ROOT_DIR, "ner_output_proj")  # models

DATA_DIR.mkdir(parents=True, exist_ok=True)
CONFIG_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Add paths in env
environ["SPACY_BINS_DIR"] = str(CONFIG_DIR)
environ["SPACY_DATA_DIR"] = str(DATA_DIR)
environ["SPACY_OUTPUT_DIR"] = str(OUTPUT_DIR)

logger.info(f"\nRoot directory: {ROOT_DIR} \nData directory: {DATA_DIR} \nConfig directory: {CONFIG_DIR} \nOutput directory: {OUTPUT_DIR}")

2025-04-21 17:48:26.657 | INFO     | __main__:<cell line: 0>:16 - 
Root directory: /content/drive/MyDrive 
Data directory: /content/drive/MyDrive/data_ner 
Config directory: /content/drive/MyDrive/config_ner 
Output directory: /content/drive/MyDrive/ner_output_proj


# Prétraitement des données NER depuis un fichier CSV :
# - Le fichier contient une colonne 'Sentence #' indiquant à quelle phrase appartient chaque mot.
# - Certaines lignes n'ont pas cette info car une phrase est indiquée une seule fois, puis les mots suivent sur plusieurs lignes.
# - On utilise `fillna(method='ffill')` pour propager l'identifiant de la phrase à toutes ses lignes.
# - Ensuite, on regroupe les lignes par phrase ('Sentence #'), pour reconstruire chaque phrase sous forme de :
#     - une liste de tokens (mots)
#     - une liste de tags NER correspondants
# - Le résultat est une liste de tuples (tokens, tags), format standard pour l'entraînement des modèles NER avec SpaCy, Flair ou Transformers.


In [ ]:
import pandas as pd

# Charger les données
base_ner = pd.read_csv("/content/drive/MyDrive/data_ner/NER_dataset.csv", encoding='utf-8')

base_ner.head(40)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


In [ ]:
# Propager les valeurs de 'Sentence #' sur les lignes NaN
base_ner['Sentence #'] = base_ner['Sentence #'].fillna(method='ffill')

base_ner.head(40)

<ipython-input-7-316ef0b04b47>:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  base_ner['Sentence #'] = base_ner['Sentence #'].fillna(method='ffill')


,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
5,Sentence: 1,through,IN,O
6,Sentence: 1,London,NNP,B-geo
7,Sentence: 1,to,TO,O
8,Sentence: 1,protest,VB,O
9,Sentence: 1,the,DT,O


In [ ]:

# Regrouper les données par phrase
sentences = []
for _, group in base_ner.groupby("Sentence #"):
    tokens = group["Word"].tolist()
    tags = group["Tag"].tolist()
    sentences.append((tokens, tags))

sentences[0:4]

[(['Thousands',
   'of',
   'demonstrators',
   'have',
   'marched',
   'through',
   'London',
   'to',
   'protest',
   'the',
   'war',
   'in',
   'Iraq',
   'and',
   'demand',
   'the',
   'withdrawal',
   'of',
   'British',
   'troops',
   'from',
   'that',
   'country',
   '.'],
  ['O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-geo',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-geo',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-gpe',
   'O',
   'O',
   'O',
   'O',
   'O']),
 (['Iranian',
   'officials',
   'say',
   'they',
   'expect',
   'to',
   'get',
   'access',
   'to',
   'sealed',
   'sensitive',
   'parts',
   'of',
   'the',
   'plant',
   'Wednesday',
   ',',
   'after',
   'an',
   'IAEA',
   'surveillance',
   'system',
   'begins',
   'functioning',
   '.'],
  ['B-gpe',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-tim',
   'O',
   'O',
   'O',
   'B-org',
   'O',
   'O',
   'O',
  

# Modéle NER avec Spacy

In [ ]:
import spacy
from spacy.tokens import DocBin
from pathlib import Path
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# On suppose que 'sentences' contient toutes les phrases du corpus au format [(tokens, tags), ...]
# Découpage des données en ensembles d'entraînement, validation et test
sentences_train, temp_sentences = train_test_split(sentences, test_size=0.3, random_state=42)  # 70% train, 30% reste
sentences_valid, sentences_test = train_test_split(temp_sentences, test_size=0.5, random_state=42)  # 15% valid, 15% test

# Récupération de la liste des étiquettes uniques (y compris le tag 'O' pour "outside")
label_list = list(set([tag for _, tags in sentences for tag in tags]))

def create_spacy_docs(sentences, label_names=None):
    nlp = spacy.blank("en")  # Crée un modèle vide
    doc_bin = DocBin()

    for tokens, tags in tqdm(sentences):
        tokens = [token for token in tokens if str(token) != 'nan']
        tags = [tag if isinstance(tag, str) else 'O' for tag in tags]

        doc = spacy.tokens.Doc(nlp.vocab, words=tokens)
        ents = []
        start = None
        end = None
        label = None

        for i, (token, tag) in enumerate(zip(doc, tags)):
            if tag.startswith("B-"):
                if start is not None:
                    # Ajoute l'entité précédente si elle existe
                    span = doc.char_span(start, end, label=label)
                    if span:
                        ents.append(span)
                start = token.idx
                end = token.idx + len(token)
                label = tag[2:]
            elif tag.startswith("I-") and label is not None:
                end = token.idx + len(token)
            else:
                if start is not None:
                    span = doc.char_span(start, end, label=label)
                    if span:
                        ents.append(span)
                    start = end = label = None

        # Dernière entité à la fin de la phrase
        if start is not None:
            span = doc.char_span(start, end, label=label)
            if span:
                ents.append(span)

        doc.ents = ents
        doc_bin.add(doc)

    return doc_bin




In [ ]:
# Supposons que ces données existent déjà sous forme de listes de tuples (tokens, tags)
# sentences_train, sentences_valid, sentences_test
# et que label_list contient les entités

# Création des fichiers d'entraînement, validation et test au format .spacy
train_db = create_spacy_docs(sentences_train, label_list)
valid_db = create_spacy_docs(sentences_valid, label_list)
test_db  = create_spacy_docs(sentences_test, label_list)

# Sauvegarde des fichiers .spacy dans le répertoire défini
train_db.to_disk(DATA_DIR / "train.spacy")
valid_db.to_disk(DATA_DIR / "valid.spacy")
test_db.to_disk(DATA_DIR / "test.spacy")

100%|██████████| 7194/7194 [00:01<00:00, 4120.54it/s]


# entrainement du modele NER avec Spacy

In [ ]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 2.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Crée un fichier config
!python -m spacy init config {CONFIG_DIR}/base_config.cfg --lang en --pipeline ner --optimize accuracy


⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: accuracy
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/config_ner/base_config.cfg
You can now add your data and train your pipeline:
python -m spacy train base_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy init fill-config ${SPACY_BINS_DIR}/base_config.cfg ${SPACY_BINS_DIR}/comp_config.cfg

⚠ Nothing to auto-fill: base config is already complete
✔ Saved config
/content/drive/MyDrive/config_ner/comp_config.cfg
You can now add your data and train your pipeline:
python -m spacy train comp_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train $SPACY_BINS_DIR/comp_config.cfg \
--paths.train $SPACY_DATA_DIR/train.spacy --paths.dev $SPACY_DATA_DIR/valid.spacy \
--output $SPACY_OUTPUT_DIR

ℹ Saving to output directory:
/content/drive/MyDrive/ner_output_proj
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     49.38    0.14    4.29    0.07    0.00
  0     200         57.67   2539.65   61.50   63.45   59.66    0.61
  0     400         79.43   1955.08   68.66   71.57   65.98    0.69
  0     600         57.29   1977.49   72.18   76.03   68.71    0.72
  0     800         64.24   2028.09   78.75   81.15   76.48    0.79
  0    1000         80.90   2389.32   79.40   78.95   79.85    0.79
  0    1200         99.67   2753.55   80.29   83.96   76.94    0.80
  0    1400        127.80   3274.11   80.95   82.39   79.56    0.81

# Optimisation du modéle NER avec Spacy

In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
!python -m spacy train $SPACY_BINS_DIR/comp_config_opt.cfg --output ${SPACY_OUTPUT_DIR} --paths.train ${SPACY_DATA_DIR}/train.spacy --paths.dev ${SPACY_DATA_DIR}/valid.spacy


ℹ Saving to output directory: /content/drive/MyDrive/ner_output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     13.24    5.35    2.96   27.61    0.10
  0     200        210.59    964.23   38.89   40.13   37.73    0.39
  0     400         39.62    625.01   55.48   57.26   53.80    0.56
  0     600         72.77    716.56   63.53   64.24   62.84    0.64
  0     800         62.82    861.35   70.52   70.56   70.49    0.71
  0    1000         79.70   1076.69   72.03   72.59   71.48    0.72
  0    1200        148.74   1128.62   73.38   73.02   73.75    0.73
  0    1400        110.45   1285.08   76.16   76.52   75.81    0.76
  0 

In [ ]:
%%bash

# performance evaluation
START=$(date '+%Y-%m-%d %H:%M:%S')
echo "Start time: $START"

printf "\n\n***************************************** Training performances\n"
python -m spacy evaluate ${SPACY_OUTPUT_DIR}/model-best $SPACY_DATA_DIR/train.spacy --output ${SPACY_OUTPUT_DIR}/train_evaluation.json

printf "\n\n***************************************** Validation performances\n"
python -m spacy evaluate ${SPACY_OUTPUT_DIR}/model-best $SPACY_DATA_DIR/valid.spacy --output ${SPACY_OUTPUT_DIR}/valid_evaluation.json

printf "\n\n***************************************** Test performances\n"
python -m spacy evaluate ${SPACY_OUTPUT_DIR}/model-best $SPACY_DATA_DIR/test.spacy --output ${SPACY_OUTPUT_DIR}/test_evaluation.json

END=$(date '+%Y-%m-%d %H:%M:%S')
echo "End time: $END"

Start time: 2025-04-21 19:52:15


***************************************** Training performances
ℹ Using CPU

================================== Results ==================================

TOK     -    
NER P   89.95
NER R   88.63
NER F   89.28
SPEED   2654 


=============================== NER (per type) ===============================

          P       R       F
per   82.80   84.88   83.83
gpe   96.54   95.30   95.91
geo   90.71   92.25   91.47
org   86.99   78.98   82.79
tim   93.41   91.15   92.26
art   81.76   43.53   56.81
nat   36.53   72.79   48.65
eve   83.33   35.55   49.83

✔ Saved results to
/content/drive/MyDrive/ner_output_proj/train_evaluation.json


***************************************** Validation performances
ℹ Using CPU

================================== Results ==================================

TOK     -    
NER P   85.66
NER R   84.13
NER F   84.89
SPEED   2718 


=============================== NER (per type) ===============================

          P  

# prédiction

In [ ]:
custom_nlp_ner = spacy.load(Path(environ["SPACY_OUTPUT_DIR"], "model-best"))

text_test = "Ousmane Sonko visited Dakar in 2020."

doc_test = custom_nlp_ner(text_test)
for ent in doc_test.ents:
    print(ent.text, ent.label_)

Ousmane Sonko per
Dakar geo
2020 tim


In [ ]:
spacy.displacy.serve(doc_test, style="ent", auto_select_port=True)

/usr/local/lib/python3.11/dist-packages/spacy/displacy/__init__.py:106: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.
